In [1]:
from splinter import Browser #sign in function
import splinter

browser = Browser('chrome')
url = "https://zh-tw.facebook.com/login/"
browser.visit(url)
username =input('username:')
password =input('password:')
search_input = browser.find_by_xpath('//*[@id="email"]')[0]
search_input.fill(username)
search_input = browser.find_by_xpath('//*[@id="pass"]')[0]
search_input.fill(password)
browser.find_by_xpath('//*[@id="loginbutton"]').first.click()

try:
    search_input = browser.find_by_xpath('//*[@id="approvals_code"]')[0]
    phonecheck = input('phonecheck_code:')
    search_input.fill(phonecheck)
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
    browser.find_by_xpath('//*[@id="u_0_0"]/div/div[2]/ul/li[2]/div/label/strong').first.click()
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
except splinter.exceptions.ElementDoesNotExist:
    pass

phonecheck_code:582633


In [16]:
def get_friends_list(url_input):#連線到某個人讀取他的朋友清單
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests
    success = 0 #checking connection is succeeded or not
    count = 0 #define how many times we have tried to connect
    times_of_trying = 3 #the limited number we want to try
    friends_list = []
    while (success == 0 and count<times_of_trying) : #if not success to connect try few times
        try:
            url = url_input + '/friends' #go to the friends page
            browser.visit(url)
            
            soup = BeautifulSoup(browser.html, 'lxml') #find how many friends the user has
            numbers_of_friends = soup.select('#pagelet_timeline_medley_friends > div._3cz > div.clearfix._1_ca')
            match = re.match('All Friends([0-9,]+).*',numbers_of_friends[0].text)
            
            a = int((int(match.group(1).replace(',','')) - 20)/6)+5 #from the amount of friends calculate scrolling times

            while a:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(0.2)
                a -= 1
                
            success = 1 #if succeeded define success connection
            
            soup = BeautifulSoup(browser.html, 'lxml') 
            section_friends = soup.select('#collection_wrapper_2356318349')[0].select('div')[0].select('ul')

            aa = 0 #just a number to show sequence of friend
            for friends in section_friends: #find the friends list and save it to friend_list
                for friend in friends.select('li'):
                    if len(friend.select('div > div > div > div > div > div > a')) != 0:
                        if friend.select('div > div > div > div > div > div > a')[0].text == 'FriendFriends':
                            friends_list.append([friend.select('div > div > div > div > div > div > a')[1].text,
                                                friend.select('div > div > div > div > div > div > a')[1]['href']])
#                             print(aa,friend.select('div > div > div > div > div > div > a')[1].text)
#                             print(friend.select('div > div > div > div > div > div > a')[1]['href'])
                        else:
                            friends_list.append([friend.select('div > div > div > div > div > div > a')[0].text,
                                                friend.select('div > div > div > div > div > div > a')[0]['href']])
#                             print(aa,friend.select('div > div > div > div > div > div > a')[0].text)
#                             print(friend.select('div > div > div > div > div > div > a')[0]['href'])
                aa+=1
        except requests.ConnectionError:
            success = 1    
        except AttributeError: #if user does not permit us to see his firends try few times
#             print('try',count,'times')
            count += 1
            if count==times_of_trying:
#                 print('already tried')
                pass
    return friends_list

In [7]:
def get_likes_list(url_input,browser):#連線到某個人抓他按讚的粉絲頁
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests

    success = 0  #checking connection is succeeded or not
    count = 0  #define how many times we have tried to connect
    likes_list = []
    
    while (success == 0 and count<4) :
        try:
            browser.visit(url_input) #go user's page and find his likes
            browser.find_by_text('More').first.click()
            browser.find_by_text('Likes').first.click()
            
            soup = BeautifulSoup(browser.html, 'lxml') #find how many likes the user has
            likes = soup.select('#pagelet_timeline_medley_likes > div._3cz > div.clearfix._1_ca > div._3dc.lfloat._ohe._5brz > a')[0].text.split()[1]
            numbers_of_likes = ''.join(re.findall('[\d]+',likes))
#             print(numbers_of_likes)
            a = int((int(numbers_of_likes.replace(',','')) - 20)/6)+10 #from the amount of likes calculate scrolling times
            while a:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(0.2)
                a -= 1
            success = 1
            
            soup = BeautifulSoup(browser.html, 'lxml') #find the likes list and save it to likes_list
            chose_link = 2 #some regular pattern that what we wand is at the second part of each block
            for link in soup.select('#collection_wrapper_2409997254')[0].select('div')[0].select('a'):
                if chose_link%3 == 0: #we get the second of each block
                    likes_list.append([link.text,link['href']])
#                     print(link.text)
#                     print(link['href'])
                chose_link += 1
            # soup.select('#collection_wrapper_2409997254')[0].select('div')[0].select('a')[1].text
        except splinter.exceptions.ElementDoesNotExist:
            print(url,'\n','user do not permit likes reading','\n','or wrong url')
            success = 1
        except Exception as ee:
            print(ee)
            count += 1
            
    return likes_list

In [135]:
import csv
url = 'https://www.facebook.com/v78203' #set the starting user
crawling_level = 0
ones_likes_list = [url,get_likes_list(url)]
ones_friends_list_firstlevel = [url,get_friends_list(url)]
with open('./data/facebook_likes.csv', 'a+') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(ones_likes_list)
for element in ones_friends_list_firstlevel[1]:
    url = element[1].replace('?fref=pb&hc_location=friends_tab','')
    ones_likes_list = [url,get_likes_list(url)]
    with open('./data/facebook_likes.csv', 'a+') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ones_likes_list)

https://www.facebook.com/kicokico26 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/george.kao2 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/etina80610 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/yushihting 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/kaiwei.tseng.9028 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/profile.php?id=100000172058437&fref=pb&hc_location=friends_tab 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/pierce.lan 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/lewis.lin.7399 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/jung.chen.5074 
 user do not apply likes reading 
 or wrong url


KeyboardInterrupt: 

In [132]:
print(ones_friends_list[1][0][1].replace('?fref=pb&hc_location=friends_tab',''))
print(ones_friends_list[1][0][1])
print(ones_friends_list[1][1][1])

https://www.facebook.com/jex.lee.9
https://www.facebook.com/jex.lee.9?fref=pb&hc_location=friends_tab
https://www.facebook.com/yhnjum?fref=pb&hc_location=friends_tab
